In [ ]:
# This does basic reductions (bias, dark, flat correction, etc.)

# Created 2021 Oct 19 by E.S.

In [54]:
from astropy.io import fits
import numpy as np
import glob
import os
import matplotlib.pyplot as plt

In [53]:
# directories of files to use

parent_stem = "/Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211018"
flat_stem = parent_stem + "/flats_for_master_flat"
light_stem = parent_stem + "/sci_data_to_reduce"
processed_stem = parent_stem + "/processed_sci_data"

master_bias_file_name = parent_stem + "/master_bias_20211019.fits"

# this 10-sec dark has bias already subtracted
master_dark_file_name = parent_stem + "/master_dark_bias_subtd_20211019.fits"

In [38]:
# read in master dark (already bias-subtracted)
master_dark_obj = fits.open(master_dark_file_name)
master_dark = master_dark_obj[0].data

# read in master bias
master_bias_obj = fits.open(master_bias_file_name)
master_bias = master_bias_obj[0].data

In [39]:
file_list_lights_src = glob.glob(light_stem + "/*fit")
file_list_flats_src = glob.glob(flat_stem + "/*fit")

In [47]:
## make a master flat

# initialize cube
cube_flats = np.nan*np.ones((512,512,len(file_list_flats_src)))

# loop over the flats, regularize, and assemble into cube
for i in range(0,len(file_list_flats_src)):
    
    this_raw_flat = fits.open(file_list_flats_src[i])

    # subtract bias from all the raw flat frames
    flat_bias_subt = np.subtract(this_raw_flat[0].data,master_bias)

    # rescale flats to be equivalent to 10-sec integration time (integration time of science frames and dark)
    # n.b. flat integration time is under keyword 'EXPTIME'
    exp_time = this_raw_flat[0].header["EXPTIME"]
    flat_bias_subt_rescaled = np.multiply((10./exp_time),flat_bias_subt)

    # subtract the 10-sec master dark (which is already bias-subtracted) from the 10-sec-equivalent flats
    flat_bias_subt_rescaled_dark_subted = np.subtract(flat_bias_subt_rescaled,master_dark)

    # put dark-subtracted flats into a cube
    cube_flats[:,:,i] = flat_bias_subt_rescaled_dark_subted

In [51]:
# take 2D median across flats, then divide by median value across that 2D array to get one master flat
median_unnormzed_flat = np.median(cube_flats, axis=2)
master_flat = np.divide(median_unnormzed_flat,np.median(median_unnormzed_flat))

In [52]:
# write out, in case

hdu = fits.PrimaryHDU(master_flat)
hdu.writeto('junk_master_flat.fits')

In [60]:
# divide lights by master flat

for j in range(0,len(file_list_lights_src)):
    
    # read in light
    light_obj = fits.open(file_list_lights_src[j])
    light = light_obj[0].data
    
    # flat-fielding
    light_ffed = np.divide(light,master_flat)
    
    # write out, while keeping same header
    write_file_name = processed_stem + "/" + os.path.basename(file_list_lights_src[j])
    hdu2 = fits.PrimaryHDU(light_ffed, header=light_obj[0].header)
    hdu2.writeto(write_file_name, overwrite=True)
    
    print("Wrote " + write_file_name)

Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211018/processed_sci_data/toi_3722-0382.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211018/processed_sci_data/toi_3722-0396.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211018/processed_sci_data/toi_3722-1088.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211018/processed_sci_data/toi_3722-1063.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211018/processed_sci_data/toi_3722-1077.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211018/processed_sci_data/toi_3722-0369.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211018/processed_sci_data/toi_3722-0341.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211018/processed_sci_data/toi_3722-0427.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211018/processed_sci_data/toi_3722-0355.fit
W

Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211018/processed_sci_data/toi_3722-0817.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211018/processed_sci_data/toi_3722-0803.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211018/processed_sci_data/toi_3722-0354.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211018/processed_sci_data/toi_3722-0340.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211018/processed_sci_data/toi_3722-0426.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211018/processed_sci_data/toi_3722-0368.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211018/processed_sci_data/toi_3722-1076.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211018/processed_sci_data/toi_3722-1062.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211018/processed_sci_data/toi_3722-1089.fit
W

Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211018/processed_sci_data/toi_3722-1465.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211018/processed_sci_data/toi_3722-1303.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211018/processed_sci_data/toi_3722-0948.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211018/processed_sci_data/toi_3722-0974.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211018/processed_sci_data/toi_3722-0960.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211018/processed_sci_data/toi_3722-0141.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211018/processed_sci_data/toi_3722-0627.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211018/processed_sci_data/toi_3722-1539.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211018/processed_sci_data/toi_3722-0633.fit
W

Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211018/processed_sci_data/toi_3722-1111.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211018/processed_sci_data/toi_3722-0569.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211018/processed_sci_data/toi_3722-1110.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211018/processed_sci_data/toi_3722-0568.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211018/processed_sci_data/toi_3722-1104.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211018/processed_sci_data/toi_3722-0232.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211018/processed_sci_data/toi_3722-0554.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211018/processed_sci_data/toi_3722-0540.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211018/processed_sci_data/toi_3722-1138.fit
W

Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211018/processed_sci_data/toi_3722-1489.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211018/processed_sci_data/toi_3722-0797.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211018/processed_sci_data/toi_3722-0783.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211018/processed_sci_data/toi_3722-0973.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211018/processed_sci_data/toi_3722-0967.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211018/processed_sci_data/toi_3722-0032.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211018/processed_sci_data/toi_3722-0754.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211018/processed_sci_data/toi_3722-0740.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211018/processed_sci_data/toi_3722-0998.fit
W

...

Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211018/processed_sci_data/toi_3722-0567.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211018/processed_sci_data/toi_3722-0201.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211018/processed_sci_data/toi_3722-0215.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211018/processed_sci_data/toi_3722-0573.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211018/processed_sci_data/toi_3722-1123.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211018/processed_sci_data/toi_3722-1137.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211018/processed_sci_data/toi_3722-0229.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211018/processed_sci_data/toi_3722-0598.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211018/processed_sci_data/toi_3722-0003.fit
W

Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211018/processed_sci_data/toi_3722-1492.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211018/processed_sci_data/toi_3722-0968.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211018/processed_sci_data/toi_3722-0983.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211018/processed_sci_data/toi_3722-1445.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211018/processed_sci_data/toi_3722-1323.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211018/processed_sci_data/toi_3722-1337.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211018/processed_sci_data/toi_3722-0029.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211018/processed_sci_data/toi_3722-0997.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211018/processed_sci_data/toi_3722-1451.fit
W

Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211018/processed_sci_data/toi_3722-0372.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211018/processed_sci_data/toi_3722-1078.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211018/processed_sci_data/toi_3722-0366.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211018/processed_sci_data/toi_3722-0400.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211018/processed_sci_data/toi_3722-0428.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211018/processed_sci_data/toi_3722-1050.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211018/processed_sci_data/toi_3722-1044.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211018/processed_sci_data/toi_3722-0819.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211018/processed_sci_data/toi_3722-1291.fit
W

Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211018/processed_sci_data/toi_3722-0171.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211018/processed_sci_data/toi_3722-1509.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211018/processed_sci_data/toi_3722-0824.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211018/processed_sci_data/toi_3722-1284.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211018/processed_sci_data/toi_3722-1290.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211018/processed_sci_data/toi_3722-0830.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211018/processed_sci_data/toi_3722-0818.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211018/processed_sci_data/toi_3722-1045.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211018/processed_sci_data/toi_3722-0429.fit
W

Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211018/processed_sci_data/toi_3722-1330.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211018/processed_sci_data/toi_3722-0990.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211018/processed_sci_data/toi_3722-1456.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211018/processed_sci_data/toi_3722-0748.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211018/processed_sci_data/toi_3722-0760.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211018/processed_sci_data/toi_3722-0006.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211018/processed_sci_data/toi_3722-1318.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211018/processed_sci_data/toi_3722-0012.fit
Wrote /Users/bandari/Documents/postdoc_notre_dame/kriz_data/data_20211018/processed_sci_data/toi_3722-0774.fit
W